In [6]:
# using python kernel 3.13.2

In [7]:
import os
import json
import subprocess
import random
import numpy as np
from pathlib import Path
from datetime import datetime, timezone
from itertools import product, combinations
from collections import defaultdict
import sys
import time


## Configuration

In [ ]:
# Seed for reproducibility
RANDOM_SEED = 2084
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# Project paths
ROOT = Path('S:/UVG/LOCKIN/emotune/workspace')
TRANSFORMER = ROOT / 'transformer'
EXP = ROOT / 'exp/eval_run_v7_01'
DATASET_ROOT = Path('S:/UVG/LOCKIN/emotune/dataset')

# Models to evaluate
MODELS = [
    {
        'name': 'scratch',
        'load_ckt': 'checkpoints',
        'load_ckt_loss': '8', # Mejor modelo propio
        'load_dict': 'train_dictionary.pkl',
        'word2event_path': DATASET_ROOT / 'word2event_scratch.json',
    },
    # {
    #     'name': 'pretrained',
    #     'load_ckt': 'checkpoints',
    #     'load_ckt_loss': 'pre',
    #     'load_dict': 'dictionary.pkl',
    #     'word2event_path': ROOT / 'word2event.json',
    # },
]

# ALL inference modes to compare
# Note: inference-normal removed as it's functionally identical to inference-primed
MODES = [
    'inference-deterministic',
    'inference-primed',
    'inference-forced'
]

# Tokens of interest for conditioning (NOTE-level tokens)
TOKENS_OF_INTEREST = ['duration', 'velocity']

# Emotions to evaluate (stratified)
EMOTIONS = [1, 2, 3, 4]
KEYS = ['A:maj', 'C:maj', 'E:min']  # Only for scratch model

# Base generation parameters
COMMON_ARGS = {
    'task_type': '4-cls',
    'num_songs': 1,  # samples per combination
    'save_tokens_json': 1,
    'generation_timeout': 30
}

# How many values to sample per token for condition grid
MAX_VALUES_PER_TOKEN = {
    'duration': 2,
    'velocity': 2,
}

# Condition set sizes to test (e.g., [1, 2] = single token and pairs)
CONDITION_SET_SIZES = [1]  # Start with single token conditioning


## HELPER FUNCTIONS

In [9]:
def load_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

def build_candidates_from_word2event(word2event):
    """Build condition candidates for tokens of interest"""
    candidates = {}
    for token in TOKENS_OF_INTEREST:
        if token not in word2event:
            continue
        items = word2event[token]
        cands = []
        for idx_str, label in items.items():
            # Skip sentinel values
            if str(idx_str) == '0' or label == 'CONTI' or label == 'Note_Duration_0':
                continue
            if isinstance(label, str):
                cands.append(label)
        if cands:
            cands.sort()  # Sort for consistency
            candidates[token] = cands
    return candidates

def sample_values_stratified(vals, n, seed_offset=0):
    """Sample n values evenly across the range"""
    rng = random.Random(RANDOM_SEED + seed_offset)
    if len(vals) <= n:
        return vals
    # Sample evenly across the range
    indices = [int(i * len(vals) / n) for i in range(n)]
    return [vals[i] for i in indices]

def build_condition_grid(model_name, candidates_map):
    """Build grid of condition combinations for a model"""
    token_names = [t for t in TOKENS_OF_INTEREST if t in candidates_map]
    
    # Sample values per token
    token_to_vals = {}
    for i, t in enumerate(token_names):
        vals = candidates_map[t]
        n = MAX_VALUES_PER_TOKEN.get(t, 3)
        token_to_vals[t] = sample_values_stratified(vals, n, seed_offset=i)
    
    # Generate combinations
    grid = []  # list of (token_subset, list_of_condition_dicts)
    for k in CONDITION_SET_SIZES:
        for subset in combinations(token_names, k):
            grids_for_subset = [token_to_vals[t] for t in subset]
            combinations_list = []
            for combo in product(*grids_for_subset):
                cond_dict = {t: v for t, v in zip(subset, combo)}
                combinations_list.append(cond_dict)
            grid.append((subset, combinations_list))
    
    return grid

def build_cmd(mode, model, emotion_tag, conditions=None, force_tokens=None, out_dir=None, key_tag=None):
    """
    Build command to run main_cp.py.
    
    Args:
        mode: inference mode string
        model: dict with model configuration
        emotion_tag: int, emotion category
        conditions: optional dict for primed mode
        force_tokens: optional dict for forced mode
        out_dir: output directory path
        key_tag: optional key tag (only for scratch model)
    """
    args = []
    def add(k, v):
        args.extend([f'--{k}', str(v)])

    add('data_root', str(DATASET_ROOT / 'co-representation'))
    add('exp_path', str(EXP))
    add('mode', mode)
    add('task_type', COMMON_ARGS['task_type'])
    add('num_songs', COMMON_ARGS['num_songs'])
    add('emo_tag', emotion_tag)
    add('save_tokens_json', COMMON_ARGS['save_tokens_json'])
    add('load_ckt', model['load_ckt'])
    add('load_ckt_loss', model['load_ckt_loss'])
    add('load_dict', model['load_dict'])
    
    # Mode-specific conditioning
    if mode in ['inference-primed', 'inference-deterministic'] and conditions:
        add('conditions', json.dumps(conditions))
    elif mode == 'inference-forced' and force_tokens:
        add('force_tokens', json.dumps(force_tokens))
    
    # Key tag for scratch model only
    if model['name'] == 'scratch' and key_tag:
        add('key_tag', key_tag)
    
    # Output directory
    if out_dir:
        add('out_dir', str(out_dir))
    
    cmd = ["python", str(TRANSFORMER / 'main_cp.py')] + args
    return cmd

## MAIN EVALUATION

In [10]:
# Tiempo de inicio
main_evaluation_start_time = time.time()

In [11]:
print('=' * 70)
print('COMPREHENSIVE EVALUATION CONFIGURATION')
print('=' * 70)
print(f"Models: {[m['name'] for m in MODELS]}")
print(f"Modes: {MODES}")
print(f"Tokens of interest: {TOKENS_OF_INTEREST}")
print(f"Emotions: {EMOTIONS}")
print(f"Condition set sizes: {CONDITION_SET_SIZES}")
print(f"Random seed: {RANDOM_SEED}")
print('=' * 70)
print()

# Load vocabularies
MODEL_TO_WORD2EVENT = {}
MODEL_TO_CANDIDATES = {}

for m in MODELS:
    name = m['name']
    vocab_path = m.get('word2event_path')
    
    if not vocab_path or not Path(vocab_path).exists():
        print(f"[!] Missing vocab for {name}: {vocab_path}")
        continue
    
    w2e = load_json(vocab_path)
    MODEL_TO_WORD2EVENT[name] = w2e
    MODEL_TO_CANDIDATES[name] = build_candidates_from_word2event(w2e)

# Build condition grids
MODEL_TO_CONDITION_GRID = {}
for m in MODELS:
    name = m['name']
    cands = MODEL_TO_CANDIDATES.get(name, {})
    MODEL_TO_CONDITION_GRID[name] = build_condition_grid(name, cands)

# Calculate total runs
total_runs = 0
for model in MODELS:
    grid = MODEL_TO_CONDITION_GRID.get(model['name'], [])
    keys_count = len(KEYS) if model['name'] == 'scratch' else 1
    for subset, cond_list in grid:
        total_runs += len(cond_list) * len(MODES) * len(EMOTIONS) * keys_count

print(f"Total runs to execute: {total_runs}")
print("=" * 70)
print()

# Main evaluation loop
ALL_RUNS = []
run_idx = 0

for model in MODELS:
    model_name = model['name']
    grid = MODEL_TO_CONDITION_GRID.get(model_name, [])
    
    print(f"{'='*70}")
    print(f"MODEL: {model_name.upper()}")
    print(f"{'='*70}")
    
    for subset, cond_list in grid:
        subset_label = '_'.join(subset) if subset else 'unconditional'
        
        for cond in cond_list:
            # Stringify condition for directory name
            cond_str = '_'.join([f"{k[:3]}{v.split('_')[-1]}" for k, v in cond.items()]) if cond else 'none'
            
            # For scratch model, iterate over keys; for pretrained, use None
            keys_to_test = KEYS if model_name == 'scratch' else [None]
            
            for emotion in EMOTIONS:
                for key in keys_to_test:
                    for mode in MODES:
                        run_idx += 1
                        
                        # Create output directory
                        stamp = datetime.now(timezone.utc).strftime('%Y%m%d-%H%M%S%f')[:19]
                        key_str = f"_key{key.replace(':', '')}" if key else ""
                        out_dir = EXP / f"{model_name}_{mode}_{subset_label}_emo{emotion}{key_str}_{cond_str}_{stamp}"
                        out_dir.mkdir(parents=True, exist_ok=True)
                        
                        # Determine conditioning based on mode
                        conditions = cond if mode in ['inference-primed', 'inference-deterministic'] else None
                        force_tokens = cond if mode in ['inference-forced'] else None
                        
                        # Build and run command
                        cmd = build_cmd(mode, model, emotion, conditions, force_tokens, out_dir, key)
                        
                        key_info = f" | key:{key}" if key else ""
                        print(f"[{run_idx}/{total_runs}] {model_name:12s} | {mode:26s} | emo{emotion}{key_info} | {cond}")
                        
                        try:
                            print('Command:\n', ' '.join(cmd))

                            proc = subprocess.run(cmd, cwd=str(TRANSFORMER), 
                                                capture_output=True, text=True, timeout=300)
                            
                            # Store metadata
                            ALL_RUNS.append({
                                'model': model_name,
                                'mode': mode,
                                'emotion': emotion,
                                'key': key,
                                'subset': list(subset),
                                'conditions': cond,
                                'out_dir': str(out_dir),
                                'returncode': proc.returncode,
                                'stdout': proc.stdout[-2000:],
                                'stderr': proc.stderr[-2000:],
                            })
                            
                            status = "✓" if proc.returncode == 0 else f"✗ (code {proc.returncode})"
                            print(f"  Status: {status}")
                            
                        except subprocess.TimeoutExpired:
                            print(f"  Status: ✗ TIMEOUT")
                            ALL_RUNS.append({
                                'model': model_name,
                                'mode': mode,
                                'emotion': emotion,
                                'key': key,
                                'subset': list(subset),
                                'conditions': cond,
                                'out_dir': str(out_dir),
                                'returncode': -1,
                                'stdout': '',
                                'stderr': 'TIMEOUT',
                            })

print("\n" + "=" * 70)
print(f"EVALUATION COMPLETE: {len(ALL_RUNS)} total runs")
print("=" * 70)

# Summary
success_count = sum(1 for r in ALL_RUNS if r['returncode'] == 0)
print(f"Successful: {success_count}/{len(ALL_RUNS)}")
print(f"Failed: {len(ALL_RUNS) - success_count}/{len(ALL_RUNS)}")
print()

# Save run metadata
runs_file = EXP / 'all_runs_metadata.json'
with open(runs_file, 'w', encoding='utf-8') as f:
    json.dump(ALL_RUNS, f, indent=2)
print(f"Run metadata saved to: {runs_file}")
print()

COMPREHENSIVE EVALUATION CONFIGURATION
Models: ['scratch', 'pretrained']
Modes: ['inference-deterministic', 'inference-primed', 'inference-forced']
Tokens of interest: ['duration', 'velocity']
Emotions: [1, 2, 3, 4]
Condition set sizes: [1]
Random seed: 2084

Total runs to execute: 192

MODEL: SCRATCH
[1/192] scratch      | inference-deterministic    | emo1 | key:A:maj | {'duration': 'Note_Duration_1080'}
Command:
 python S:\UVG\LOCKIN\emotune\workspace\transformer\main_cp.py --data_root S:\UVG\LOCKIN\emotune\dataset\co-representation --exp_path S:\UVG\LOCKIN\emotune\workspace\exp\eval_run_v7_01 --mode inference-deterministic --task_type 4-cls --num_songs 1 --emo_tag 1 --save_tokens_json 1 --load_ckt checkpoints --load_ckt_loss 8 --load_dict train_dictionary.pkl --conditions {"duration": "Note_Duration_1080"} --key_tag A:maj --out_dir S:\UVG\LOCKIN\emotune\workspace\exp\eval_run_v7_01\scratch_inference-deterministic_duration_emo1_keyAmaj_dur1080_20251103-0601416989
  Status: ✓
[2/192] 

In [12]:
# Tiempo de fin
main_evaluation_end_time = time.time()
print(f"Tiempo total de evaluación: {main_evaluation_end_time - main_evaluation_start_time:.2f} segundos")

Tiempo total de evaluación: 4938.34 segundos


## ADHERENCE ANALYSIS

In [13]:
# Tiempo de inicio
adherence_analysis_start_time = time.time()

In [14]:
print("=" * 70)
print("ADHERENCE ANALYSIS (with distance metrics)")
print("=" * 70)
print()

# Import the updated adherence function
sys.path.insert(0, str(TRANSFORMER))
from analyze_adherence import compute_adherence
from analyze_emotion_key import compute_emotion_key_adherence

REPORTS = []
EMOTION_KEY_REPORTS = []

# Token adherence analysis
# Note: Skip token adherence for inference-forced mode since forced tokens
# will always be 100% adherent by design (tautological measurement).
# We still compute emotion/key adherence for forced mode to assess whether
# constraining note-level tokens disrupts higher-level musical structure.
for run in ALL_RUNS:
    if run['returncode'] != 0:
        continue
    
    # Skip token adherence for forced mode (measure only emotion/key)
    if run['mode'] == 'inference-forced':
        continue
    
    out_dir = run['out_dir']
    model_name = run['model']
    
    # Find metadata files
    metas = [f for f in os.listdir(out_dir) if f.endswith('.meta.json')]
    if not metas:
        continue
    
    # Get word2event for this model
    word2event = MODEL_TO_WORD2EVENT.get(model_name)
    
    per_file = []
    for mf in metas:
        meta_path = os.path.join(out_dir, mf)
        try:
            with open(meta_path, 'r', encoding='utf-8') as f:
                meta = json.load(f)
            
            tokens = np.load(meta['paths']['npy'])
            conditions = meta.get('conditions_indices') or meta.get('force_indices') or {}
            token_order = meta.get('vocab_order') or []
            
            # Compute adherence with distance metrics
            adherence = compute_adherence(tokens, conditions, token_order, word2event)
            
            per_file.append({
                'meta': meta_path,
                'adherence': adherence,
                'num_tokens': tokens.shape[0]
            })
            
        except Exception as e:
            print(f"  [ERROR] {mf}: {e}")
            continue
    
    if per_file:
        REPORTS.append({'run': run, 'files': per_file})

print(f"Analyzed token adherence for {len(REPORTS)} successful runs")
print()

# Emotion and key adherence analysis
print("=" * 70)
print("EMOTION AND KEY ADHERENCE ANALYSIS")
print("=" * 70)
print()

for run in ALL_RUNS:
    if run['returncode'] != 0:
        continue
    
    out_dir = run['out_dir']
    emotion = run['emotion']
    key = run.get('key')  # Only for scratch model
    
    # Skip if directory doesn't exist
    if not os.path.exists(out_dir):
        continue
    
    print(f"Analyzing: {os.path.basename(out_dir)}")
    
    try:
        results = compute_emotion_key_adherence(out_dir, emotion, key)
        
        EMOTION_KEY_REPORTS.append({
            'run': run,
            'emotion_adherence': results.get('emotion', {}),
            'key_adherence': results.get('key', {}) if key else None,
            'num_files': results.get('num_files', 0)
        })
        
        # Print summary
        if 'emotion' in results and 'adherence_ratio' in results['emotion']:
            emo_adh = results['emotion']['adherence_ratio'] * 100
            print(f"  Emotion adherence: {emo_adh:.1f}% ({results['emotion']['matches']}/{results['emotion']['total']})")
        
        if key and 'key' in results and 'adherence_ratio' in results['key']:
            key_adh = results['key']['adherence_ratio'] * 100
            print(f"  Key adherence: {key_adh:.1f}% ({results['key']['matches']}/{results['key']['total']})")
            if 'key_distribution' in results['key']:
                print(f"  Key distribution: {results['key']['key_distribution']}")
        
    except Exception as e:
        print(f"  [ERROR] {e}")
        continue

print(f"\nAnalyzed emotion/key for {len(EMOTION_KEY_REPORTS)} runs")
print()


ADHERENCE ANALYSIS (with distance metrics)

Analyzed token adherence for 128 successful runs

EMOTION AND KEY ADHERENCE ANALYSIS

Analyzing: scratch_inference-deterministic_duration_emo1_keyAmaj_dur1080_20251103-0601416989
  Key adherence: 0.0% (0/1)
  Key distribution: {'A:min': 1}
Analyzing: scratch_inference-primed_duration_emo1_keyAmaj_dur1080_20251103-0602057929
  Emotion adherence: 0.0% (0/1)
  Key adherence: 0.0% (0/1)
  Key distribution: {'D:maj': 1}
Analyzing: scratch_inference-forced_duration_emo1_keyAmaj_dur1080_20251103-0602219801
  Emotion adherence: 0.0% (0/1)
  Key adherence: 0.0% (0/1)
  Key distribution: {'D:maj': 1}
Analyzing: scratch_inference-deterministic_duration_emo1_keyCmaj_dur1080_20251103-0603274911
  Emotion adherence: 100.0% (1/1)
  Key adherence: 0.0% (0/1)
  Key distribution: {'A:min': 1}
Analyzing: scratch_inference-primed_duration_emo1_keyCmaj_dur1080_20251103-0603397479
  Emotion adherence: 0.0% (0/1)
  Key adherence: 0.0% (0/1)
  Key distribution: {'A:

In [15]:
# Tiempo de fin
adherence_analysis_end_time = time.time()
print(f"Tiempo total de análisis de adherencia: {adherence_analysis_end_time - adherence_analysis_start_time:.2f} segundos")

Tiempo total de análisis de adherencia: 4177.34 segundos


## AGGREGATE RESULTS

In [16]:
# ============================================================================
# AGGREGATE RESULTS
# ============================================================================

print("=" * 70)
print("AGGREGATING RESULTS: COMPARISON ACROSS INFERENCE MODES")
print("=" * 70)
print()

# Aggregate by (model, mode, emotion, key_tag, token_subset)
agg = defaultdict(lambda: defaultdict(lambda: {'adherence': [], 'distance': []}))
emotion_key_agg = defaultdict(lambda: {'emotion_adherence': [], 'key_adherence': [], 'key_correlation': []})

# Aggregate token adherence
for report in REPORTS:
    run = report['run']
    key = (run['model'], run['mode'], run['emotion'], run.get('key'), tuple(run['subset']))
    
    # Average per file, then accumulate
    for f in report['files']:
        for token_name, stats in f['adherence'].items():
            agg[key][token_name]['adherence'].append(stats['adherence_ratio'])
            if stats.get('is_numeric') and 'mean_distance' in stats:
                agg[key][token_name]['distance'].append(stats['mean_distance'])

# Aggregate emotion/key adherence
for report in EMOTION_KEY_REPORTS:
    run = report['run']
    key = (run['model'], run['mode'], run['emotion'], run.get('key'), tuple(run['subset']))
    
    if 'emotion_adherence' in report and isinstance(report['emotion_adherence'], dict):
        if 'adherence_ratio' in report['emotion_adherence']:
            emotion_key_agg[key]['emotion_adherence'].append(report['emotion_adherence']['adherence_ratio'])
    
    if report.get('key_adherence') and isinstance(report['key_adherence'], dict):
        if 'adherence_ratio' in report['key_adherence']:
            emotion_key_agg[key]['key_adherence'].append(report['key_adherence']['adherence_ratio'])
        if 'mean_correlation' in report['key_adherence']:
            emotion_key_agg[key]['key_correlation'].append(report['key_adherence']['mean_correlation'])

# Compute final statistics
# Include entries from both token adherence (agg) and emotion/key adherence (emotion_key_agg)
# This ensures forced mode entries are included (with only emotion/key metrics)
all_keys = set(agg.keys()) | set(emotion_key_agg.keys())

final_results = []
for key in all_keys:
    model, mode, emotion, key_tag, subset = key
    row = {
        'model': model,
        'mode': mode,
        'emotion': emotion,
        'key': key_tag,
        'subset': list(subset),
    }
    
    # Add token adherence metrics (if available - not present for forced mode)
    if key in agg:
        token_stats = agg[key]
        for token_name, metrics in token_stats.items():
            if metrics['adherence']:
                row[f'{token_name}_adherence'] = float(np.mean(metrics['adherence']))
            if metrics['distance']:
                row[f'{token_name}_distance'] = float(np.mean(metrics['distance']))
    
    # Add emotion/key adherence if available
    if key in emotion_key_agg:
        ek_metrics = emotion_key_agg[key]
        if ek_metrics['emotion_adherence']:
            row['emotion_adherence'] = float(np.mean(ek_metrics['emotion_adherence']))
        if ek_metrics['key_adherence']:
            row['key_adherence'] = float(np.mean(ek_metrics['key_adherence']))
        if ek_metrics['key_correlation']:
            row['key_correlation'] = float(np.mean(ek_metrics['key_correlation']))
    
    final_results.append(row)

# Save results
results_file = EXP / 'evaluation_results_comprehensive.json'
with open(results_file, 'w', encoding='utf-8') as f:
    json.dump(final_results, f, indent=2)

print(f"Results saved to: {results_file}")
print(f"Total result rows: {len(final_results)}")
print()


AGGREGATING RESULTS: COMPARISON ACROSS INFERENCE MODES

Results saved to: S:\UVG\LOCKIN\emotune\workspace\exp\eval_run_v7_01\evaluation_results_comprehensive.json
Total result rows: 96



## DISPLAY RESULTS: COMPARISON TABLE

In [17]:
print("=" * 90)
print("ADHERENCE COMPARISON BY MODE (Average across emotions and conditions)")
print("=" * 90)
print()

# Group by (model, mode) and average everything else
mode_comparison = defaultdict(lambda: defaultdict(lambda: {'adherence': [], 'distance': []}))

for row in final_results:
    key = (row['model'], row['mode'])
    for k, v in row.items():
        if k.endswith('_adherence'):
            token = k.replace('_adherence', '')
            mode_comparison[key][token]['adherence'].append(v)
        elif k.endswith('_distance'):
            token = k.replace('_distance', '')
            mode_comparison[key][token]['distance'].append(v)
        elif k == 'key_correlation':
            # Store correlation as a "distance" metric for key (higher = better)
            mode_comparison[key]['key']['distance'].append(v)

# Print comparison table
all_tokens = TOKENS_OF_INTEREST + ['emotion', 'key']

for model_name in [m['name'] for m in MODELS]:
    print(f"{'─' * 90}")
    print(f"Model: {model_name.upper()}")
    print(f"{'─' * 90}")
    print(f"{'Mode':26s} | {'Token':10s} | {'Adherence %':12s} | {'Avg Distance':12s}")
    print(f"{'-' * 90}")
    
    for mode in MODES:
        key = (model_name, mode)
        if key not in mode_comparison:
            continue
        
        first = True
        for token in all_tokens:
            if token not in mode_comparison[key]:
                continue
            
            metrics = mode_comparison[key][token]
            adherence_pct = np.mean(metrics['adherence']) * 100 if metrics['adherence'] else 0.0
            distance = np.mean(metrics['distance']) if metrics['distance'] else None
            
            # Skip key for pretrained model
            if token == 'key' and model_name == 'pretrained':
                continue
            
            mode_str = mode if first else ''
            # For key, distance is correlation (higher = better), for others it's actual distance (lower = better)
            if token == 'key' and distance is not None:
                distance_str = f"{distance:.4f} ↑"  # Arrow indicates higher is better
            elif distance is not None:
                distance_str = f"{distance:.2f}"
            else:
                distance_str = 'N/A'
            
            print(f"{mode_str:26s} | {token:10s} | {adherence_pct:11.2f}% | {distance_str:>12s}")
            first = False
        
        if not first:  # Add separator between modes
            print()

print("=" * 90)
print()

ADHERENCE COMPARISON BY MODE (Average across emotions and conditions)

──────────────────────────────────────────────────────────────────────────────────────────
Model: SCRATCH
──────────────────────────────────────────────────────────────────────────────────────────
Mode                       | Token      | Adherence %  | Avg Distance
------------------------------------------------------------------------------------------
inference-deterministic    | duration   |        8.05% |       677.59
                           | velocity   |        1.36% |        23.52
                           | emotion    |       45.83% |          N/A
                           | key        |       41.67% |     0.7444 ↑

inference-primed           | duration   |        7.14% |       762.87
                           | velocity   |        1.78% |        25.52
                           | emotion    |       35.42% |          N/A
                           | key        |       33.33% |     0.7556 ↑

inference